<a href="https://colab.research.google.com/github/Haarinaath-Y/Namelessscoder/blob/main/Data_Streaming_into_Spark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Init Job

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget https://dlcdn.apache.org/spark/spark-3.4.2/spark-3.4.2-bin-hadoop3.tgz
!tar -xvf spark-3.4.2-bin-hadoop3.tgz
!pip install -q findspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.4.2-bin-hadoop3"
import findspark
findspark.init()

In [ ]:
# prompt: install google generative ai
!pip install -q -U google-generativeai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.4/137.4 kB 642.9 kB/s eta 0:00:00


In [ ]:
!pip install langchain_google_genai

In [ ]:
!pip install slack_sdk

In [ ]:
!pip install langchain

In [ ]:
!pip install chromadb

In [ ]:
!pip install langchain_experimental

Test Data

In [ ]:
def get_data_test():
    import requests
    test_res=requests.get('https://randomuser.me/api/')
    test_res = test_res.json()
    test_res = test_res['results'][0]
    return test_res

get_data_test()

# Streaming Data from API

In [ ]:
def get_data():
    import requests
    res=requests.get('https://randomuser.me/api/')
    res = res.json()
    res = res['results'][0]
    return res

def format_data(res):
    data={}
    location=res['location']
    data['first_name'] = res['name']['first']
    data['last_name'] = res['name']['last']
    data['gender'] = res['gender']
    data['address'] = f"{str(location['street']['number'])} {location['street']['name']}, " \
                      f"{location['city']}, {location['state']}, {location['country']}"
    data['city']=location['city']
    data['country']=location['country']
    data['dob']=res['dob']['date']
    data['registered_on']=res['registered']['date']
    data['email'] = res['email']
    data['phone'] = res['phone']

    return data

def stream_data():
    import json
    result = get_data()
    result = format_data(result)
    return result

def repeat(parameter):
    return [stream_data() for i in range(parameter)]

streamed_data = repeat(1000)

To pull 1000 requests, it took 4m 18s on first run

2-5 minutes to pull 1000 requests



In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import IntegerType,FloatType,DoubleType
from pyspark.sql.types import StructType, StructField, ArrayType, StringType

In [ ]:
spark = SparkSession.builder \
    .appName("StreamDatatoSpark") \
    .getOrCreate()

In [ ]:
streamed_data_extract = [dict(kv_pair.items()) for kv_pair in streamed_data]

In [ ]:
print(streamed_data_extract)

[{'first_name': 'Abhijith', 'last_name': 'Keshri', 'gender': 'male', 'address': '6734 Racecource Rd, Pali, Manipur, India', 'city': 'Pali', 'country': 'India', 'dob': '1945-04-01T12:16:14.067Z', 'registered_on': '2010-09-05T01:04:18.143Z', 'email': 'abhijith.keshri@example.com', 'phone': '7744589218'}, {'first_name': 'Lot', 'last_name': 'Van de Watering', 'gender': 'female', 'address': '6983 Het Duin, Voorhout, Limburg, Netherlands', 'city': 'Voorhout', 'country': 'Netherlands', 'dob': '1960-07-31T20:23:47.400Z', 'registered_on': '2003-09-29T03:20:05.924Z', 'email': 'lot.vandewatering@example.com', 'phone': '(070) 9120395'}, {'first_name': 'Anujna', 'last_name': 'Almeida', 'gender': 'female', 'address': '7741 Commercial St, Rewa, Dadra and Nagar Haveli, India', 'city': 'Rewa', 'country': 'India', 'dob': '1993-07-28T10:17:44.310Z', 'registered_on': '2011-01-06T23:42:43.441Z', 'email': 'anujna.almeida@example.com', 'phone': '8302640025'}, {'first_name': 'Đura', 'last_name': 'Nemanjić', '

In [ ]:
stream_data_rdd = spark.sparkContext.parallelize(streamed_data_extract)

In [ ]:
stream_data_df = spark.createDataFrame(stream_data_rdd)

In [ ]:
stream_data_df.show()

+--------------------+-------------------+--------------+--------------------+--------------------+-----------+------+---------------+--------------+--------------------+
|             address|               city|       country|                 dob|               email| first_name|gender|      last_name|         phone|       registered_on|
+--------------------+-------------------+--------------+--------------------+--------------------+-----------+------+---------------+--------------+--------------------+
|6734 Racecource R...|               Pali|         India|1945-04-01T12:16:...|abhijith.keshri@e...|   Abhijith|  male|         Keshri|    7744589218|2010-09-05T01:04:...|
|6983 Het Duin, Vo...|           Voorhout|   Netherlands|1960-07-31T20:23:...|lot.vandewatering...|        Lot|female|Van de Watering| (070) 9120395|2003-09-29T03:20:...|
|7741 Commercial S...|               Rewa|         India|1993-07-28T10:17:...|anujna.almeida@ex...|     Anujna|female|        Almeida|    8302640

In [ ]:
stream_data_df.printSchema()

root
 |-- address: string (nullable = true)
 |-- city: string (nullable = true)
 |-- country: string (nullable = true)
 |-- dob: string (nullable = true)
 |-- email: string (nullable = true)
 |-- first_name: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- phone: string (nullable = true)
 |-- registered_on: string (nullable = true)



In [ ]:
import pandas as pd
pandas_df = stream_data_df.toPandas()
file_output=pandas_df.to_csv("dataextract.csv")
file_read=pd.read_csv("dataextract.csv")

In [ ]:
import os
from builtins import open

with open('SlackAPI.txt','r') as f:
  lines= f.readlines()

SLACK_API_KEY = lines[0].strip()

In [ ]:
import logging, os

# Sets the debug level.
# If you're using this in production, you can change this back to INFO and add extra log entries as needed.
logging.basicConfig(level=logging.DEBUG)

# Initialize the Web API client.
# This expects that you've already set your SLACK_BOT_TOKEN as an environment variable.
# Try to resist the urge to put your token directly in your code; it is best practice not to.
from slack_sdk import WebClient
client = WebClient(SLACK_API_KEY)

In [ ]:
# Tests to see if the token is valid
auth_test = client.auth_test()
bot_user_id = auth_test["user_id"]
print(f"App's bot user: {bot_user_id}")


App's bot user: U06RWG53FN1


In [ ]:
auth_test = client.auth_test()

In [ ]:
print(auth_test)

{'ok': True, 'url': 'https://awrfikghost.slack.com/', 'team': 'Awrfik Ghost', 'user': 'alertsbot', 'team_id': 'T06RWFPRY93', 'user_id': 'U06RWG53FN1', 'bot_id': 'B06SN7J6P1N', 'is_enterprise_install': False}


In [ ]:
files = client.files_list(user=bot_user_id)

In [ ]:
# Get a list of all channels the authenticated user is in
channel_list = client.conversations_list()

# Iterate through the channels and find the one with the name "alert"
for channel in channel_list["channels"]:
    if channel["name"] == "alerts":
        alert_channel_id = channel["id"]
        break

print(alert_channel_id)

C06RZDHPNJW


In [ ]:
from datetime import date
today = date.today()
print(today)

2024-04-01


In [ ]:
new_file = client.files_upload_v2(
    title="Today's Data Extract File - {}".format(today),
    file="dataextract.csv",
    initial_comment="Hi there! This is a text file!",
)

In [ ]:
file_url = new_file.get("file").get("permalink")
new_message = client.chat_postMessage(
    channel=alert_channel_id,
    text=f"Here is the file: {file_url}",
)

In [ ]:
import pathlib
import textwrap

import google.generativeai as genai

# Used to securely store your API key
from google.colab import userdata

from IPython.display import display
from IPython.display import Markdown


def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [ ]:
from builtins import open

with open('GeminiAPI.txt','r') as f:
  lines= f.readlines()

In [ ]:
GOOGLE_API_KEY = lines[0].strip()
genai.configure(api_key=GOOGLE_API_KEY)
os.environ['GOOGLE_API_KEY']=GOOGLE_API_KEY

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.prompts import PromptTemplate
from langchain.chains.question_answering import load_qa_chain
from langchain.document_loaders.csv_loader import CSVLoader
from langchain_google_genai import ChatGoogleGenerativeAI
from google.generativeai.types import HarmCategory, HarmBlockThreshold

In [ ]:
from langchain.agents.agent_types import AgentType
from langchain_experimental.agents import create_csv_agent
import pandas as pd

In [ ]:
insight_generate_model = ChatGoogleGenerativeAI(model='gemini-pro',temperature=0.9,safety_settings={
                HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
                HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
                HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
                HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE
            })

In [ ]:
question_agent = "How many Indians are in the list? \nWhat's the country with nighest no of people? \nWhat's the most common name and he/she is from which country?"

In [ ]:
agent=create_csv_agent(llm=insight_generate_model,path=r"/content/dataextract.csv",verbose=True,agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,handle_parsing_errors=True)
prompt_template = "You have given a data of users with details to answer the question and make sure that you provide all the details\n Question: {}\n Answer: ".format(question_agent)
result = agent.run(prompt_template)

/usr/local/lib/python3.10/dist-packages/langchain_experimental/agents/agent_toolkits/pandas/base.py:242: UserWarning: Received additional kwargs {'handle_parsing_errors': True} which are no longer supported.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...
Thought: I need to first find the Indian users, then check the countries with highest number of users, and lastly check the most common name and the country of the person with the name.
Action: python_repl_ast
Action Input: df[df['country'] == 'India'].count()[0]49There are 49 Indians in the list.
Action: python_repl_ast
Action Input: df['country'].value_counts().idxmax()IranIran is the country with the highest number of people (100) in the list.
Action: python_repl_ast
Action Input: df['first_name'].value_counts().idxmax()JuliaJulia is the most common name.
Action: python_repl_ast
Action Input: df[df['first_name'] == 'Julia']['country'].value_counts().idxmax()FinlandJulia is from Finland.
Final Answer: **1. There are 49 Indians in the list. 2. Iran is the country with highest no of people (100) 3. Most common name is Julia and she is from Finland.**

> Finished chain.


In [ ]:
print(result)

**1. There are 49 Indians in the list. 2. Iran is the country with highest no of people (100) 3. Most common name is Julia and she is from Finland.**


In [ ]:
post_insights = client.chat_postMessage(
    channel=alert_channel_id,
    text=f"Here are some insights from today's data: \n{result}",
)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')